### Import dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
df = pd.read_csv("games.csv") # Import als df, Komma als seperator
df.head(3)

20058

### Einfache Beschreibung des Datensatzes

In [19]:
print('Anzahl der Spiele:', len(df))
print('Davon gewertete Spiele:', len(df[df["rated"] == True]), '(entspricht', 100 * round(len(df[df["rated"] == True])/len(df), 4) ,'%)')
print()
print('Minimale Anzahl gezogener Züge:', df["turns"].min())
print('Maximale Anzahl gezogener Züge:', df["turns"].max())
print('Durchschnittliche Anzahl gezogener Züge:', df["turns"].mean())
print()
print('Partieausgang:')
print('Schachmatt:', 100 * round(len(df[df["victory_status"] == "mate"])/len(df), 4), '%')
print('Aufgabe:', 100 * round(len(df[df["victory_status"] == "resign"])/len(df), 4), '%')
print('Ablauf der Zeit:', 100 * round(len(df[df["victory_status"] == "outoftime"])/len(df), 4), '%')
print('Remis:', 100 * round(len(df[df["victory_status"] == "draw"])/len(df), 4), '% (Remis bei Ablauf der Zeit hier nicht mit eingerechnet)')
print()
print('Anteil der gewonnen Spiele:') 
print('Weiß:', 100 * round(len(df[df["winner"] == "white"])/len(df), 4), '%')
print('Schwarz:', 100 * round(len(df[df["winner"] == "black"])/len(df), 4), '%')
print('Remis:', 100 * round(len(df[df["winner"] == "draw"])/len(df), 4), '%')
print()
# Unterteile Zeitbegrenzung in Bullet(1-2), Blitz(3-7), Rapid(8-22) und Classical(23-180), ignoriere Inkremente
# Erstelle zunächst Variable mit Zeitbegrenzung ohne Inkrement als Integer
# df["time_control"] = np.nan
# for i in range(len(df)):
#     df["time_control"][i] = int(df["increment_code"][i].partition("+")[0])
print('Zeitbegrenzung der Spiele:') 
print('Bullet (1-2 Minuten):', 100 * round(len(df[df["time_control"] <= 2])/len(df), 4), '%')
print('Blitz (3-7 Minuten):', 100 * round(len(df[(df["time_control"] >= 3) & (df["time_control"] <= 7)])/len(df), 4), '%')
print('Rapid (8-20 Minuten):', 100 * round(len(df[(df["time_control"] >= 8) & (df["time_control"] <= 22)])/len(df), 4), '%')
print('Classical (25-180 Minuten):', 100 * round(len(df[df["time_control"] >= 23])/len(df), 4), '%')
print()
print('Anzahl Spieler:', len(df["white_id"].unique()) + len(df["black_id"].unique()))
print('Durschnittliche Anzahl Spiele pro Spieler:',  round((2 * len(df)) / (len(df["white_id"].unique()) + len(df["black_id"].unique())), 2))
print()
print('Kleinste Spielerwertung (ELO):', min(df[["white_rating", "black_rating"]].min()))
print('Größte Spielerwertung:', max(df[["white_rating", "black_rating"]].max()))
print('Durchschnittliche Spielerwertung:', round(sum(df[["white_rating", "black_rating"]].sum()) / (2 *len(df))))
print()
print('Erster Zug:')
df["first_move"] = np.nan
for i in range(len(df)):
    df["first_move"][i] = df["moves"][i].partition(' ')[0]
for i in df["first_move"].unique():
    print(i + ':', round(100 * len(df[df["first_move"] == i])/len(df), 2), '%', )
print()
print('Gespielte Eröffnungen:')
for i in df["opening_name"].unique():
    if len(df[df["opening_name"] == i])/len(df) > 0.01:
        print(i + ':', round(100 * len(df[df["opening_name"] == i])/len(df), 2), '%', )


# Evtl. hier noch Eröffnungszug und Eröffnung

Anzahl der Spiele: 20058
Davon gewertete Spiele: 16155 (entspricht 80.54 %)

Minimale Anzahl gezogener Züge: 1
Maximale Anzahl gezogener Züge: 349
Durchschnittliche Anzahl gezogener Züge: 60.46599860404826

Partieausgang:
Schachmatt: 31.53 %
Aufgabe: 55.57 %
Ablauf der Zeit: 8.38 %
Remis: 4.52 % (Remis bei Ablauf der Zeit hier nicht mit eingerechnet)

Anteil der gewonnen Spiele:
Weiß: 49.86 %
Schwarz: 45.4 %
Remis: 4.74 %

Zeitbegrenzung der Spiele:
Bullet (1-2 Minuten): 0.65 %
Blitz (3-7 Minuten): 13.73 %
Rapid (8-20 Minuten): 77.62 %
Classical (25-180 Minuten): 7.99 %

Anzahl Spieler: 18769
Durschnittliche Anzahl Spiele pro Spieler: 2.14

Kleinste Spielerwertung (ELO): 784
Größte Spielerwertung: 2723
Durchschnittliche Spielerwertung: 1593

Erster Zug:
d4: 22.54 %
e4: 62.81 %
e3: 2.07 %
c4: 3.57 %
Nf3: 3.61 %
a4: 0.14 %
Nh3: 0.07 %
g3: 0.93 %
h4: 0.16 %
f4: 0.83 %
b4: 0.44 %
Nc3: 0.49 %
b3: 0.86 %
h3: 0.07 %
d3: 0.65 %
f3: 0.11 %
g4: 0.19 %
a3: 0.13 %
c3: 0.28 %
Na3: 0.02 %

Gespielte

In [15]:
df["first_move"] = np.nan
for i in range(len(df)):
    df["first_move"][i] = df["moves"][i].partition(' ')[0]
for i in df["first_move"].unique():
    print(i + ':', round(100 * len(df[df["first_move"] == i])/len(df), 2), '%', )


d4: 22.54 %
e4: 62.81 %
e3: 2.07 %
c4: 3.57 %
Nf3: 3.61 %
a4: 0.14 %
Nh3: 0.07 %
g3: 0.93 %
h4: 0.16 %
f4: 0.83 %
b4: 0.44 %
Nc3: 0.49 %
b3: 0.86 %
h3: 0.07 %
d3: 0.65 %
f3: 0.11 %
g4: 0.19 %
a3: 0.13 %
c3: 0.28 %
Na3: 0.02 %


### Now depending on rating
- Work in Progress

In [3]:
dfr = df[df["rated"]] # Create DF with only rated games
dfr["avg_rating"] = (dfr["white_rating"] + dfr["black_rating"])/2 # Erstelle Variable mit durchscnittlichem Rating beider Spieler pro Spiel
# dfr["avg_rating"].hist()
print("Min rating:", dfr["avg_rating"].min())
print("Max rating:", dfr["avg_rating"].max())
cont = (dfr["avg_rating"].max() - dfr["avg_rating"].min())/5 # Unterteilung für Spielstärken, 5 Bereiche

Min rating: 816.5
Max rating: 2475.5


0.013909661980257254

In [5]:
for i in range(1,6): 
     print(dfr["avg_rating"].min() + (i-1)*cont, "to", dfr["avg_rating"].min() + i*cont, "White:", dfr[dfr["avg_rating"] < dfr["avg_rating"].min() + i*cont][dfr["winner"] == "white"].count()[0]/len(dfr[dfr["avg_rating"] < dfr["avg_rating"].min() + i*cont]))
     print(dfr["avg_rating"].min() + (i-1)*cont, "to", dfr["avg_rating"].min() + i*cont, "Black:", dfr[dfr["avg_rating"] < dfr["avg_rating"].min() + i*cont][dfr["winner"] == "black"].count()[0]/len(dfr[dfr["avg_rating"] < dfr["avg_rating"].min() + i*cont]))
     print(dfr["avg_rating"].min() + (i-1)*cont, "to", dfr["avg_rating"].min() + i*cont, "Draw:", dfr[dfr["avg_rating"] < dfr["avg_rating"].min() + i*cont][dfr["winner"] == "draw"].count()[0]/len(dfr[dfr["avg_rating"] < dfr["avg_rating"].min() + i*cont]))


816.5 to 1148.3 White: 0.4826989619377163
816.5 to 1148.3 Black: 0.47058823529411764
816.5 to 1148.3 Draw: 0.04671280276816609
1148.3 to 1480.1 White: 0.5104748603351955
1148.3 to 1480.1 Black: 0.45006983240223464
1148.3 to 1480.1 Draw: 0.03945530726256983
1480.1 to 1811.9 White: 0.5026851998104565
1480.1 to 1811.9 Black: 0.4566419207076291
1480.1 to 1811.9 Draw: 0.04067287948191439
1811.9 to 2143.7 White: 0.49952335557673977
1811.9 to 2143.7 Black: 0.45764219891960595
1811.9 to 2143.7 Draw: 0.04283444550365428
2143.7 to 2475.5 White: 0.4984523956914696
2143.7 to 2475.5 Black: 0.4571004085675375
2143.7 to 2475.5 Draw: 0.04444719574099294


### En passant stuff
- Check for each game, if en passant was possible or happened!
- En passant possible for white, when white has a pawn on 5th rank and black moves from 7th to 5th with a pawn on a lane next to the lane of the white pawn 

In [5]:
# TO DO:
# Problem: Wenn ein weißer Bauer vom 5ten Rang wieder weggezogen ist und dann, 
# bevor schwarz auf den 5ten Rang zieht, dort eine andere Figure geschlafen wird, 
# geht das Programm davon aus, dass der weiße Bauer dort geschlagen wurde. 
# Das Programm zählt also "Weißer Bauer hingezogen" = 1, "Weißer Bauer weggezogen = 1",
# "Weißer Bauer geschlagen = 1", es müsste aber nur "Weißer Bauer hingezogen" = 1" und 
# "Weißer Bauer weggezogen = 1" sein. Nachdem der Bauer weggezogen ist, darf das Programm
# nicht mehr nach SChlagzügen suchen. 
# Problemspiel: 158

# Import regular expression module
import re 
g = 158
# Game moves:
ex = df["moves"][g]
# print('Game ' + str(g+1) + ':')
# Search for move [a-h]5 (will find pawn moves from 7th or 6th rank to 5th rank)
ah5 = [m.start() for m in re.finditer(' [a-h]5', ex)]
for i in ah5: 
    moved = False
    # Check, if it was black's move
    if ex[0:i+1].count(' ')%2 != 0:
        # Check, if pawn came from 7th or 6th rank
        # Search for moves to 6th rank on current file before current move
        ah6 = [m.start() for m in re.finditer(' ' + ex[i+1] + '6', ex[0:i])]
        if ah6:         
            for j in ah6: 
                if ex[0:j+1].count(' ')%2 != 0:
                    moved = True
                    # print(ex[i+1:i+3] + ': Black moved here from 6th rank, no captures on 6th rank')
                    break
            if moved == True:
                continue
        # Search for captures to 6th rank on current file before current move
        ahx = [m.start() for m in re.finditer(' [' + chr(ord(ex[i+1]) - 1) + chr(ord(ex[i+1]) + 1) + ']x' + ex[i+1] + '6', ex[0:i])]
        if ahx:
            # If we found captures, check, if pawn moved again to fifth rank (move or capture)
            ah55 = [m.start() for m in re.finditer(' ' + ex[i+1] + '5|' + ex[i+1] + 'x[a-h]5', ex[0:i])]
            # Check if it was blacks moves
            count = 0
            for j in ah55:
                if ex[0:j+1].count(' ')%2 != 0:
                    count = count + 1
            # If the no. of black pawns that moved to 6th rank by capture is equal to the no. of pawns that left that square, the original black pawn must have come from the starting posotion
            if len(ahx) != count:
                # print(ex[i+1:i+3] + ': Black pawn moved here from 6th rank after it captured')
                continue
        #     else: 
        #         print(ex[i+1:i+3] + ': Black moved here from starting position, black pawns on 6th rank moved away again')     
        # print(ex[i+1:i+3] + ': Black moved here from starting position')  

        # Now check, if white pawn is on 5th rank
        # Moves TO adjacent squares on fifth rank, loop for left and right adjacent square:
        for j in [-1, 1]:
            # Search for pawn moves or captures to adjacent square before black's move
            ah5l = [m.start() for m in re.finditer(' ' + chr(ord(ex[i+1]) + j) + '5|[a-h]x' + chr(ord(ex[i+1]) + j) + '5', ex[0:i])]
            if ah5l:
                # Check for each case, if it was white's move:
                tol = 0 # Count times that white moved to the square
                white_move = np.zeros(len(ah5l))
                for k in range(0, len(ah5l)):
                    if ex[0:ah5l[k]+1].count(' ')%2 == 0:
                        white_move[k] = True # Was it white's move?
                        tol = tol + 1
                # Create Vector with all white pawn moves to the square
                ah5lw = ah5l * white_move
                ah5lw = ah5lw[ah5lw != 0]

                # Moves FROM adjacent squares on fifth rank:
                # Search for pawn moves or captures from adjacent square before black's move:
                ah6l = [m.start() for m in re.finditer(' ' + chr(ord(ex[i+1]) + j) + '6|' + chr(ord(ex[i+1]) + j) + 'x[a-h]6', ex[0:i])]
                # Check for each case, if it was white's move
                froml = 0 # Count times that white moved to the square
                for k in ah6l:
                    if ex[0:k+1].count(' ')%2 == 0:
                        froml = froml + 1

                # Captures of pawns that moved to specific square
                for k in range(0, len(ah5lw)):
                    space = re.search(' ', ex[int(ah5lw[k])+1:len(ex)-1]) # Start searching AFTER the white move was played  
                    if k < len(ah5lw) - 1: # Search for capture in between white moves to specific square
                        capture = re.search('x' + chr(ord(ex[i+1]) + j) + '5', ex[int(ah5lw[k]) + 1 + space.start() : int(ah5lw[k+1])])
                    else: # Do this, if it's the last move from white to the square
                        capture = re.search('x' + chr(ord(ex[i+1]) + j) + '5', ex[int(ah5lw[k]) + 1 + space.start() : len(ex) - 1])
                    if capture != None: # If we find a capture, remove one from the counts "white pawn moved to specific square"
                        tol = tol - 1

                # If no. of moves to the square equals no. of moves from the square, it is empty, otherwise moves to the square should be 1 greater than moves from the square, so the square is occupied by a pawn and... EN PASSANT POSSIBLE!
                if tol != froml:
                    print('Game ' + str(g) + ': ' + ex[i+1:i+3] + ': En passant possible!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    print(ex)
                    break
            #     else:
            #         if tol == 0:
            #             print(ex[i+1:i+3] + ': No en passant possible: There was never a white pawn on square ' + str(j) + ' or it was captured')
            #         else:
            #             print(ex[i+1:i+3] + ': No en passant possible: The white pawns moved away from square ' + str(j) + ' or it was captured')
            # else: 
            #     print(ex[i+1:i+3] + ': No en passant possible: There was never a pawn on square ' + str(j))

Game 1:
d5: Black moved here from starting position
d5: No en passant possible: There was never a pawn on square -1
d5: No en passant possible: There was never a pawn on square 1
Game 2:
e5: Black moved here from starting position
e5: No en passant possible: There was never a pawn on square -1
e5: No en passant possible: There was never a pawn on square 1
Game 3:
e5: Black moved here from starting position
e5: No en passant possible: There was never a pawn on square -1
e5: No en passant possible: There was never a pawn on square 1
b5: Black moved here from starting position
b5: No en passant possible: There was never a pawn on square -1
b5: No en passant possible: There was never a pawn on square 1
a5: Black moved here from starting position
a5: No en passant possible: There was never a pawn on square -1
a5: No en passant possible: There was never a white pawn on square 1
c5: Black moved here from 6th rank, no captures on 6th rank
d5: Black moved here from 6th rank, no captures on 6th 